## 初始化Api

In [ ]:
import json

import requests

url = "http://energyscene.zenergyai.com:38080/api/"
payload = json.dumps({"username": "zenergy@zenergy.ai", "password": "ZEnergy.Ai2312"})
response = requests.post(f"{url}auth/login", data=payload)
token = response.json()["token"]

## 导入设备

In [ ]:
import pandas as pd

df = pd.read_csv('../data/ThingsBoardDevice.csv')
df.set_index('ProjectId', inplace=True)
df

## 获取服务端属性

In [ ]:
def get_attribute(device_id: str, key: str) -> str:
    """
    获取属性
    :param device_id: 设备ID
    :param key: 属性键
    :return: 属性值
    """
    entity_type = "DEVICE"
    attribute_response = requests.get(
        f"{url}plugins/telemetry/{entity_type}/{device_id}/values/attributes?keys={key}",
        headers={"X-Authorization": f"Bearer {token}"})
    return attribute_response.json()[0]["value"]

## 获取设备属性

In [ ]:
# 获取服务端属性
df['Eb-Active'] = df['Eb-DeviceId'].apply(lambda x: get_attribute(x, 'active'))
df['Eb-LastActivityTime'] = df['Eb-DeviceId'].apply(lambda x: get_attribute(x, 'lastActivityTime'))
df['Ew-Active'] = df['Ew-DeviceId'].apply(lambda x: get_attribute(x, 'active'))
df['Ew-LastActivityTime'] = df['Ew-DeviceId'].apply(lambda x: get_attribute(x, 'lastActivityTime'))
# 转换时间
df['Eb-LastActivityTime'] = (pd.to_datetime(df['Eb-LastActivityTime'], unit='ms', utc=True)
                             .dt.tz_convert('Asia/Shanghai'))
df['Ew-LastActivityTime'] = (pd.to_datetime(df['Ew-LastActivityTime'], unit='ms', utc=True)
                             .dt.tz_convert('Asia/Shanghai'))
# 列排序
df = df[['Eb-Active', 'Ew-Active', 'Eb-LastActivityTime', 'Ew-LastActivityTime', 'Eb-DeviceId', 'Ew-DeviceId']]
df = df.sort_values(by=['Eb-Active', 'Ew-Active'], ascending=True)
df

## 统计

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# 统计在线离线数量
eb_offline_count = df[df['Eb-Active'] == False].shape[0]
eb_online_count = df[df['Eb-Active'] == True].shape[0]
ew_offline_count = df[df['Ew-Active'] == False].shape[0]
ew_online_count = df[df['Ew-Active'] == True].shape[0]
# 绘制饼图
fig = make_subplots(rows=1, cols=2, subplot_titles=("EB", "EW"), specs=[[{"type": "domain"}, {"type": "domain"}]])
fig.add_trace(go.Pie(labels=["在线", "离线"], values=[eb_online_count, eb_offline_count], name="EB"), 1, 1)
fig.add_trace(go.Pie(labels=["在线", "离线"], values=[ew_online_count, ew_offline_count], name="EW"), 1, 2)
fig.show()